In [1]:
from data_prep import *
from charts import *
import pandas as pd

game_df = team_sheets()
players_df = players(game_df)
players_agg_df = players_agg(players_df)
lineouts_df = lineouts()
pitchero_df = pitchero_stats()
set_piece_df = set_piece_results()
analysis = game_stats()

/Users/samlindsay/Documents/Projects/Personal/RugbyResults/env/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


# Season Summaries

In [2]:
seasons = [
    "2021/22", 
    "2022/23", 
    "2023/24", 
    "2024/25"
]

for season in seasons:
    print(season)

    # Starts (by Position)
    plot_starts_by_position(1, season=season, fb_only=False, df=players_df).save(f"Charts/{season.replace('/','-')}/1s-squad-position.html")
    plot_starts_by_position(2, season=season, fb_only=False, df=players_df).save(f"Charts/{season.replace('/','-')}/2s-squad-position.html")
    # Appearances
    plot_games_by_player(0, min=10, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/appearances.html")
    plot_games_by_player(1, min=5, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/1s-appearances.html")
    plot_games_by_player(2, min=5, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/2s-appearances.html")
    # Points Scorers
    points_scorers_chart(1, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/1s-points.html")
    points_scorers_chart(2, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/2s-points.html")
    # Captains
    if season != ["2021/22", "2022/23"]:
        captains_chart(season, df=game_df).save(f"Charts/{season.replace('/','-')}/captains.html")
    # Cards
    card_chart(0, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/cards.html")
    # Results
    results_chart(0, season, df=game_df).save(f"Charts/{season.replace('/','-')}/results.html")
    # results_chart(1, season, df=game_df).save(f"Charts/{season.replace('/','-')}/1s-results.html")
    # results_chart(2, season, df=game_df).save(f"Charts/{season.replace('/','-')}/2s-results.html")
    
    # Set Piece
    set_piece_h2h_charts(1, season, df=set_piece_df).save(f"Charts/{season.replace('/','-')}/1s-set-piece.html")
    set_piece_h2h_charts(2, season, df=set_piece_df).save(f"Charts/{season.replace('/','-')}/2s-set-piece.html")

    # # Lineouts
    lineout_chart(1, season, df=lineouts_df).save(f"Charts/{season.replace('/','-')}/1s-lineouts.html")
    if season != "2021/22":
        lineout_chart(2, season, df=lineouts_df).save(f"Charts/{season.replace('/','-')}/2s-lineouts.html")

2021/22


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:802: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:882: SettingWithCopyWarning: 
A value is trying to be set o

2022/23


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:802: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:882: SettingWithCopyWarning: 
A value is trying to be set o

2023/24


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:802: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:882: SettingWithCopyWarning: 
A value is trying to be set o

2024/25


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:802: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:882: SettingWithCopyWarning: 
A value is trying to be set o

# Trends


In [3]:
plot_games_by_player(0, min=10, season=None, agg=True, df=players_df).save("Charts/by-season/appearances.html")
captains_chart(season=None, df=game_df).save("Charts/by-season/captains.html")
card_chart(season=None, df=pitchero_df).save("Charts/by-season/cards.html")
points_scorers_chart(squad=0, season=None, df=pitchero_df).save("Charts/by-season/points.html")
results_chart(squad=0, season=None, df=game_df).save("Charts/by-season/results.html")

# Set Piece

In [4]:
set_piece_h2h_charts(1, season=None, df=set_piece_df).save("Charts/by-season/set-piece-h2h.html")

# 1st XV Game Analysis

In [8]:
game_stats_charts(analysis).save("Charts/by-season/game-stats.html")

/Users/samlindsay/Documents/Projects/Personal/RugbyResults/env/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:1427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penalties.loc[:,"stack_order"] = penalties.loc[:,"Metric"].apply(lambda x: penalty_cols.index(x))
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:1429: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

pitchero_df

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
0,Dave Bridges,2022/23,1st,14,D Bridges,15.0,4.0,0.0,0.0,0.0,1.0,0.0,20.0,1.333333,20.0,0.0,0.0
1,Josh Brimecombe,2022/23,1st,15,J Brimecombe,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Guy Collins,2024/25,1st,13,G Collins,13.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0,0.769231,10.0,0.0,0.0
3,George Naylor,2023/24,2nd,9,G Naylor,8.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.625000,5.0,0.0,0.0
4,Michael Presland,2024/25,2nd,8,M Presland,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,Josh Weaver,2023/24,1st,1,J Weaver,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
519,Colm Geraghty,2024/25,2nd,1,C Geraghty,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.000000,5.0,0.0,0.0
520,Connor Williams,2024/25,2nd,1,C Williams,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.000000,5.0,0.0,0.0
521,Darren Gwynn,2023/24,2nd,1,D Gwynn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
def results_chart(squad=1, season=None, df=None):

    if df is None:
        df = team_sheets()

    if season is not None:
        df = df[df["Season"]==season]
    if squad != 0:
        df = df[df["Squad"]==("1st" if squad==1 else "2nd")]

    df["loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
    df["winner"] = df.apply(lambda x: x["PF"] if x["Result"] == "W" else x["PA"], axis=1)

    selection = alt.selection_point(fields=['Result'], bind='legend')
    base = alt.Chart(df).encode(
        y=alt.Y(
            'GameID:N', 
            sort=None, 
            axis=alt.Axis(
                title=None, 
                offset=15, 
                grid=False, 
                ticks=False, 
                domain=False, 
                # labelExpr="split(datum.value,'-__-')[1]"
            )
        ),
        color=alt.Color(
            'Result:N', 
            scale=alt.Scale(domain=['W', 'L'], range=['#146f14', '#981515']), 
            legend=alt.Legend(offset=20, orient="bottom", title="Click to highlight", titleOrient="left")
        ),
        opacity=alt.condition(
            selection, 
            # alt.condition(team_filter, alt.value(1), alt.value(0.2)), 
            alt.value(1),
            alt.value(0.2)
        )
    )

    bar = base.mark_bar(point=True).encode(
        x=alt.X('PF:Q', title="Points", axis=alt.Axis(orient='bottom', offset=5)),
        x2='PA:Q'
    ).properties(height=alt.Step(20), width=400)

    loser = base.mark_text(align='right', dx=-2, dy=0, color='black').encode(
        x=alt.X('loser:Q', title=None, axis=alt.Axis(orient='top', offset=5)),
        text='loser:N',
    )

    winner = base.mark_text(align='left', dx=2, dy=0, color='black').encode(
        x=alt.X('winner:Q', title=None, axis=alt.Axis(orient='top', offset=5)),
        text='winner:N',
    )

    chart = (bar + loser + winner).add_params(selection, team_filter)

    if squad == 0 or season is None:
        chart = (
            chart
            .facet(
                row=alt.Row(
                    'Squad:N', 
                    header=alt.Header(title=None, labelExpr="datum.value + ' XV'", labelFontSize=48),
                    sort=alt.SortOrder('ascending')
                ),
                column=alt.Column(
                    'Season:N', 
                    header=alt.Header(title=None, labelFontSize=40), 
                    sort=alt.SortOrder('ascending')
                )
            )
            .resolve_scale(y='independent')
        )
    
    chart = chart.properties(
        title=alt.Title(
            text=f"{('1st XV ' if squad==1 else ('2nd XV ' if squad==2 else ''))}Results {season if season is not None else '(since 2021)'}",
            subtitle=[
                "Match scores visualised by winning margin. Small bars reflect close games, colour reflects the result.",
                "Click the legend to highlight wins or losses. Click a bar to highlight results against that team."  
            ],
            offset=20
        )
    )

    return chart

results_chart(0, df=game_df)#.save("Charts/2024-25/results.html")

alt.FacetChart(...)

# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets(squad=1) 

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)